In [1]:
import cv2
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

C:\Users\tharu\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
img = cv2.imread("colorpic.jpg")

In [10]:
clicked = False
r = g = b = xpos = ypos = 0

data = pd.read_csv('colors.csv')

In [11]:
data.head()

,color,color_name,hex,R,G,B
0,air_force_blue_raf,Air Force Blue (Raf),#5d8aa8,93,138,168
1,air_force_blue_usaf,Air Force Blue (Usaf),#00308f,0,48,143
2,air_superiority_blue,Air Superiority Blue,#72a0c1,114,160,193
3,alabama_crimson,Alabama Crimson,#a32638,163,38,56
4,alice_blue,Alice Blue,#f0f8ff,240,248,255


In [12]:
x = data.iloc[:,3:6].values
y = data[['hex','color']].values

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=100)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train)

y_pred = knn.predict(x_test)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
from sklearn.preprocessing import LabelEncoder
lf = LabelEncoder()
data['hexa'] = data['hex']
data['hex'] = lf.fit_transform(data['hex'])

data.head()

,color,color_name,hex,R,G,B,hexa
0,air_force_blue_raf,Air Force Blue (Raf),189,93,138,168,#5d8aa8
1,air_force_blue_usaf,Air Force Blue (Usaf),13,0,48,143,#00308f
2,air_superiority_blue,Air Superiority Blue,227,114,160,193,#72a0c1
3,alabama_crimson,Alabama Crimson,350,163,38,56,#a32638
4,alice_blue,Alice Blue,596,240,248,255,#f0f8ff


In [18]:
x = data.iloc[:,3:6].values
y = data['hex'].values

model = Sequential()
model.add(Dense(12, input_shape=(x.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(x, y, epochs=150, batch_size=10)

Epoch 1/150
87/87 [==============================] - 4s 8ms/step - loss: -4150.9912 - accuracy: 0.0012
Epoch 2/150
87/87 [==============================] - 1s 8ms/step - loss: -5686.3975 - accuracy: 0.0023
Epoch 3/150
87/87 [==============================] - 0s 4ms/step - loss: -5724.1528 - accuracy: 0.0023
Epoch 4/150
87/87 [==============================] - 1s 7ms/step - loss: -5864.9487 - accuracy: 0.0023
Epoch 5/150
87/87 [==============================] - 1s 8ms/step - loss: -5868.3306 - accuracy: 0.0023
Epoch 6/150
87/87 [==============================] - 1s 7ms/step - loss: -5868.3291 - accuracy: 0.0023
Epoch 7/150
87/87 [==============================] - 0s 5ms/step - loss: -5868.3291 - accuracy: 0.0023
Epoch 8/150
87/87 [==============================] - 0s 5ms/step - loss: -5868.3281 - accuracy: 0.0023
Epoch 9/150
87/87 [==============================] - 1s 6ms/step - loss: -5868.3291 - accuracy: 0.0023
Epoch 10/150
87/87 [==============================] - 1s 7ms/step - loss:

In [61]:
def draw_function(event, x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global b,g,r,xpos,ypos, clicked
        clicked = True
        xpos = x
        ypos = y
        b,g,r = img[y,x]
        b = int(b)
        g = int(g)
        r = int(r)

cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_function)



while(1):

    cv2.imshow("image",img)
    if (clicked):
        out = round(model.predict([[r,g,b]])[0,0])
        color_name = data[data['hex']==out]['color'].values[0]

        cv2.rectangle(img,(20,20), (1000,60), (b,g,r), -1)
        
        cv2.rectangle(img,(20,20), (1000,60), (b,g,r), -1)

        
        text = "The model predicts as "  + color_name + ' R='+ str(r) +  ' G='+ str(g) +  ' B='+ str(b)
        
        
        cv2.putText(img, text,(50,50),2,0.8,(255,255,255),2,cv2.LINE_AA)

        
        if(r+g+b>=600):
            cv2.putText(img, text,(50,50),2,0.8,(0,0,0),2,cv2.LINE_AA)
            
        clicked=False
        

    #Break the loop when user hits 'esc' key    
    if cv2.waitKey(20) & 0xFF ==27:
        break
    
cv2.destroyAllWindows()



1/1 [==============================] - 0s 75ms/step
